<a href="https://colab.research.google.com/github/diego40g/data_science_python/blob/numpy/Copia_de_Colab_y_Big_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de Datos con Google Colab y Big Query

![image.png](https://res.cloudinary.com/startup-grind/image/upload/c_scale,w_2560/c_crop,h_640,w_2560,y_1.0_mul_h_sub_1.0_mul_640/c_crop,h_640,w_2560/c_fill,dpr_2.0,f_auto,g_center,q_auto:good/v1/gcs/platform-data-goog/event_banners/DF23-Bevy-BannerBG-Red_HxB5Gvl.png)


<a href="https://www.linkedin.com/in/bryam-vega/"><img src="https://img.shields.io/badge/LinkedIn-blue?style=for-the-badge&logo=linkedin&logoColor=white" alt="LinkedIn Badge"></a>

---

## Introducción

Este taller tiene como objetivo enseñar a los participantes cómo utilizar Google Colab, una plataforma basada en la nube para ejecutar código de Python, en combinación con BigQuery, el servicio de análisis de datos de Google, para realizar análisis y consultas de datos a gran escala de manera eficiente.

# 1. Autenticacion a la plataforma Google Cloud

In [ ]:
from google.colab import auth
auth.authenticate_user()

MessageError: ignored

In [ ]:
project_id = 'dev-fest-big-query'

# 2. Conectarse al servicio Big Query

In [ ]:
from google.cloud import bigquery

In [ ]:
client = bigquery.Client(project=project_id)

# 3. Acceder al dataset (publico o privado)

In [ ]:
dataset_ref = client.dataset(dataset_id="movielens",project="dev-fest-big-query")
dataset = client.get_dataset(dataset_ref)

# 4. Listar tablas del dataset

In [ ]:
tables = [i.table_id for i in client.list_tables(dataset)]
print("*"*40,"TABLAS DEL DATASET","*"*40)
print(tables)

**************************************** TABLAS DEL DATASET ****************************************
['movies', 'rating']


# 5. Verificando el esquema del dataset

In [ ]:
table_ref = dataset_ref.table('rating')
table = client.get_table(table_ref)

In [ ]:
for property in table.schema:
  print('Property: [','name: ', property.name, ', type: ', property.field_type,', isNull: ',property.is_nullable,']')

Property: [ name:  userId , type:  INTEGER , isNull:  True ]
Property: [ name:  movieId , type:  INTEGER , isNull:  True ]
Property: [ name:  rating , type:  FLOAT , isNull:  True ]
Property: [ name:  timestamp , type:  INTEGER , isNull:  True ]


# 6. Mostrando los datos en un dataframe (pandas)

In [ ]:
df_rating = client.list_rows(table).to_dataframe()

In [ ]:
type(df_rating)

pandas.core.frame.DataFrame

In [ ]:
df_rating.head()

,userId,movieId,rating,timestamp
0,1,1219,2.0,964983393
1,1,2253,2.0,964981775
2,1,2338,2.0,964983546
3,1,2389,2.0,964983094
4,1,2617,2.0,964982588


# 7. Utilizando consultas nativas en bigquery con google colab

In [ ]:
sql_query = """
          SELECT r.userId, r.movieId, r.rating, m.title, m.genres
          FROM `dev-fest-big-query.movielens.rating` r
          INNER JOIN `dev-fest-big-query.movielens.movies` m ON m.movieId = r.movieId
          WHERE ARRAY_LENGTH(SPLIT(m.genres, '|')) > 2
          LIMIT 1000;
"""

In [ ]:
query_job = client.query(sql_query)

In [ ]:
results = query_job.to_dataframe()

In [ ]:
results

,userId,movieId,rating,title,genres
0,1,2338,2.0,I Still Know What You Did Last Summer (1998),Horror|Mystery|Thriller
1,1,2389,2.0,Psycho (1998),Crime|Horror|Thriller
2,1,2617,2.0,"Mummy, The (1999)",Action|Adventure|Comedy|Fantasy|Horror|Thriller
3,2,114060,2.0,The Drop (2014),Crime|Drama|Thriller
4,3,2105,2.0,Tron (1982),Action|Adventure|Sci-Fi
...,...,...,...,...,...
995,182,3265,2.0,Hard-Boiled (Lat sau san taam) (1992),Action|Crime|Drama|Thriller
996,182,3753,2.0,"Patriot, The (2000)",Action|Drama|War
997,182,3981,2.0,Red Planet (2000),Action|Sci-Fi|Thriller
998,182,3986,2.0,"6th Day, The (2000)",Action|Sci-Fi|Thriller


# 8. Caso practico de un analisis de datos

Para este caso práctico vamos a utilizar la columna de generos de la tabla movies para realizar unas pequeñas queries que nos podrian dar alguna informacióin interesante

## 8.1 Cantidad de peliculas por genero

Lo que queremos saber en esta ocasión, es conocer la cantidad de películas que existen por genero, ya que como podemos apreciar en el conjunto de datos tenemos un pelicula con 3 géneros y queremos saber de forma independiente cuantas peliculas hay por esos géneros.

Nuestra consulta SQL sería:

```sql
WITH MoviesWithIndividualGenres AS (
  SELECT
    r.userId,
    r.movieId,
    r.rating,
    m.title,
    genre AS individual_genre
  FROM
    `dev-fest-big-query.movielens.rating` r
  INNER JOIN
    `dev-fest-big-query.movielens.movies` m ON m.movieId = r.movieId
  CROSS JOIN
    UNNEST(SPLIT(m.genres, '|')) AS genre
)

SELECT
  individual_genre,
  COUNT(*) AS movie_count
FROM
  MoviesWithIndividualGenres
GROUP BY
  individual_genre
HAVING
  COUNT(*) > 2
ORDER BY
  movie_count DESC
LIMIT
  1000;
```

Vamos a ejecutarla para ver si nos trae el resultado que queremos:

In [ ]:
query = """
WITH MoviesWithIndividualGenres AS (
  SELECT
    r.userId,
    r.movieId,
    r.rating,
    m.title,
    genre AS individual_genre
  FROM
    `dev-fest-big-query.movielens.rating` r
  INNER JOIN
    `dev-fest-big-query.movielens.movies` m ON m.movieId = r.movieId
  CROSS JOIN
    UNNEST(SPLIT(m.genres, '|')) AS genre
)

SELECT
  individual_genre,
  COUNT(*) AS movie_count
FROM
  MoviesWithIndividualGenres
GROUP BY
  individual_genre
HAVING
  COUNT(*) > 2
ORDER BY
  movie_count DESC
LIMIT
  1000;
"""

In [ ]:
query_job = client.query(query)
df_genre_count = query_job.to_dataframe()
df_genre_count.head(5)

## 8.2 Generos con mejor promedio de rating

Como parte de nuestro super rápido análisis, queremos conocer los generos que tienen el mejor promedio en puntuaciones de rating, para saber cuales son los mejores generos votados por los usuarios

Nuestra consulta SQL sería:

```sql
WITH MoviesWithIndividualGenres AS (
  SELECT
    r.userId,
    r.movieId,
    r.rating,
    m.title,
    genre AS individual_genre
  FROM
    `dev-fest-big-query.movielens.rating` r
  INNER JOIN
    `dev-fest-big-query.movielens.movies` m ON m.movieId = r.movieId
  CROSS JOIN
    UNNEST(SPLIT(m.genres, '|')) AS genre
)

SELECT
  individual_genre,
  COUNT(*) AS movie_count,
  AVG(rating) AS average_rating
FROM
  MoviesWithIndividualGenres
GROUP BY
  individual_genre
HAVING
  COUNT(*) > 2
ORDER BY
  average_rating DESC
LIMIT
  1000;
```

In [ ]:
query = """
WITH MoviesWithIndividualGenres AS (
  SELECT
    r.userId,
    r.movieId,
    r.rating,
    m.title,
    genre AS individual_genre
  FROM
    `dev-fest-big-query.movielens.rating` r
  INNER JOIN
    `dev-fest-big-query.movielens.movies` m ON m.movieId = r.movieId
  CROSS JOIN
    UNNEST(SPLIT(m.genres, '|')) AS genre
)

SELECT
  individual_genre,
  COUNT(*) AS movie_count,
  AVG(rating) AS average_rating
FROM
  MoviesWithIndividualGenres
GROUP BY
  individual_genre
HAVING
  COUNT(*) > 2
ORDER BY
  average_rating DESC
LIMIT
  1000;
"""

In [ ]:
query_job = client.query(query)
df_genre_rating_avg = query_job.to_dataframe()
df_genre_rating_avg.head(5)

# 9. Grafiquemos uno nuestros análisis de datos con plotly

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.express as px

In [ ]:
df_fig = df_genre_count[:7]
title = 'There are twice (2x) as many drama-related films as adventure films.  <br>' \
        '<span style="font-size:13px">There are more scary movies than romance movies.</span>' \
                  '<br><span style="font-size:10px"><i>Question:'\
                  'Get number of movies by genre </span></i>'
fig = go.Figure(data=go.Bar(x=df_fig.individual_genre, y=df_fig.movie_count, marker=dict(
            color=['#004b66','#e2e2e2','#e2e2e2','#e2e2e2','#e2e2e2','#e2e2e2','#e2e2e2']
        )))
fig.update_layout(xaxis_title='Genre of movie ', yaxis_title='Number of movies',
                  title_text = title ,margin_t = 100, showlegend = False,plot_bgcolor = 'white', hovermode='closest',width = 1000,
            height=500)

Para conocer mas sobre como hacer gráficos de calidad, accede a este artículo: https://medium.com/@vegabryam40/create-graphics-in-plotly-like-a-pro-f578f8f4fb6a